In [1]:
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from fastai.data.all import get_image_files
from fastai.vision.all import load_learner,Image, PILImage

/home/ubuntu/.virtualenvs/magnetic-tiles-defect-clR5l9w5/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJ_PATH = Path.cwd().parent
DATASET_NAME = 'MAGNETIC_TILE_SURFACE_DEFECTS'
DATASET_PATH = PROJ_PATH/'data'/DATASET_NAME

In [3]:
def get_mask_path(img_path): 
    msk_path = Path(str(img_path.parent).replace('_images', '_masks'))/f'{img_path.stem}.png'
    return msk_path

In [4]:
model_dir = PROJ_PATH/'models'
pickle_fpath = model_dir/'model_pickle_fastai.pkl'
learn = load_learner(pickle_fpath, cpu=True)

In [5]:
def dice_coef(y_true, y_pred, eps=0.02):
    intersection = np.sum(y_true * y_pred)
    if (np.mean(y_true)<eps) and (np.mean(y_pred)<eps):
        return 1
    return (2*intersection) / (np.sum(y_true) + np.sum(y_pred))

In [6]:
sz=224

def resize_and_crop_center(im, sz=sz):
    new_shape = (np.array(im.shape)/np.min(im.shape)*sz).astype(int)
    im = im.resize((new_shape[1], new_shape[0]))
    h, w = im.shape
    left = (w - sz)/2
    top = (h - sz)/2
    right = (w + sz)/2
    bottom = (h + sz)/2
    # Crop the center of the image
    im = im.crop((left, top, right, bottom))
    return im

In [7]:
test_img_path = Path.cwd().parent/'data'/'MAGNETIC_TILE_SURFACE_DEFECTS'/'test_images'
test_mask_path = Path.cwd().parent/'data'/'MAGNETIC_TILE_SURFACE_DEFECTS'/'test_masks'

test_images = get_image_files(test_img_path)

dice_list = []
for img_path in tqdm(test_images):
    mask_path = test_mask_path/f'{img_path.stem}.png'
    im = Image.open(img_path)
    mask_im = Image.open(mask_path)
    im_resized = resize_and_crop_center(im, sz=sz)
    y_true = resize_and_crop_center(mask_im, sz=sz)
    y_true = np.array(y_true)
    y_pred, *_ = learn.predict(PILImage(im_resized))
    y_pred = np.array(y_pred)
    dice = dice_coef(y_true, y_pred)
    dice_list.append(dice)

dice_arr = np.array(dice_list)
dice_arr.mean()


  0%|          | 0/78 [00:00<?, ?it/s]

/home/ubuntu/.virtualenvs/magnetic-tiles-defect-clR5l9w5/lib/python3.8/site-packages/fastai/metrics.py:370: RuntimeWarning: Mean of empty slice
  return np.nanmean(binary_dice_scores)
  1%|▏         | 1/78 [00:00<00:53,  1.43it/s]

  3%|▎         | 2/78 [00:01<00:55,  1.37it/s]

  4%|▍         | 3/78 [00:01<00:45,  1.64it/s]

  5%|▌         | 4/78 [00:02<00:40,  1.84it/s]

...

 96%|█████████▌| 75/78 [00:35<00:01,  2.19it/s]

 97%|█████████▋| 76/78 [00:36<00:00,  2.08it/s]

 99%|█████████▊| 77/78 [00:36<00:00,  2.14it/s]

100%|██████████| 78/78 [00:37<00:00,  2.09it/s]


0.8531209074273287